<a href="https://colab.research.google.com/github/linesn/llm_security/blob/main/check_hf_hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check the HuggingFace Hashes for a model
This notebook is intended to help you check the
hash values to ensure that the model you download matches the repository.

In [2]:
import os
import hashlib
import requests
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
def calculate_sha256(file_path):
  """Report the hash of the file.
  """
  sha256_hash = hashlib.sha256()
  with open(file_path, "rb") as infile:
    chunk_size = 4096
    while True:
      chunk = infile.read(chunk_size)
      if not chunk:
        break
      sha256_hash.update(chunk)
  return sha256_hash.hexdigest()


def get_and_check_file(filename, prefix):
  """Download, size, and hash the file.
  """
  address = prefix + filename
  r = requests.get(address)
  with open(filename, "wb") as f:
    f.write(r.content)
  hash = calculate_sha256(filename)
  size = os.path.getsize(filename)
  #os.remove(filename)
  return hash, size

Go check the repository for the files you plan to download. You will need to write each one you intend to check in the `hffiles` list below. You will need to set the `hfprefix` to the download address **specific to the branch you are downloading from!**

In [28]:
hffiles = [
#  "pytorch_model-00001-of-00002.bin",
#  "pytorch_model-00002-of-00002.bin",
  "adapt_tokenizer.py",
  "attention.py",
  "blocks.py",
  "config.json",
  "configuration_mpt.py",
#  "custom_embedding.py",
#  "flash_attn_triton.py",
  "generation_config.json",
  "hf_prefixlm_converter.py",
  "meta_init_context.py",
  "modeling_mpt.py",
  "norm.py",
  "param_init_fns.py",
#  "requirements.txt",
  "special_tokens_map.json",
  "tokenizer.json",
  "tokenizer_config.json",
]
hfprefix = "https://huggingface.co/mosaicml/mpt-7b-instruct/resolve/bd1748ec173f1c43e11f1973fc6e61cb3de0f327/"
# https://huggingface.co/mosaicml/mpt-7b-instruct/blob/main/

HuggingFace provides the hashes for big binary files, so you can just write those below and trust those, or you can include the big binaries in the `hffiles` list too and check them by hand (note that this latter approach might exhaust your memory on Colab).

In [8]:
ReportedHashes = {
    "pytorch_model-00001-of-00002.bin":"0693d4371bfecf4260145f0735708846d5e0ededeb91d535d8370962342cef4e",
    "pytorch_model-00002-of-00002.bin":"13197cfe0fc856d6079651e997868c9253886059c4aaad3f42ea5699a65e4cde",
}

In [31]:
ReportedSizes = {
    "pytorch_model-00001-of-00002.bin": 0,
    "pytorch_model-00002-of-00002.bin": 0,
}

In [29]:
hashes = dict()
sizes = dict()
for filename in tqdm(hffiles):
  print(filename)
  hashes[filename], sizes[filename]  = get_and_check_file(filename, hfprefix)

  0%|          | 0/14 [00:00<?, ?it/s]

adapt_tokenizer.py
attention.py
blocks.py
config.json
configuration_mpt.py
generation_config.json
hf_prefixlm_converter.py
meta_init_context.py
modeling_mpt.py
norm.py
param_init_fns.py
special_tokens_map.json
tokenizer.json
tokenizer_config.json


Check if the hashes agree.

In [32]:
for hash in ReportedHashes:
  if hash in hashes:
    print(hash, "matches" if hashes[hash]==ReportedHashes[hash] else "fails")
  else:
    hashes[hash] = ReportedHashes[hash]
    sizes[hash] = ReportedSizes[hash]

Export the results.

In [33]:
df = pd.DataFrame()
df["File"] = hashes.keys()
df["SHA256"] = hashes.values()
df["Sizes"] = sizes.values()
df

,File,SHA256,Sizes
0,adapt_tokenizer.py,1a877c5856cbea1fee9d986b5fee1889ca14ccba765f01...,1752
1,attention.py,ace86f3e715b709f4ec0c44179d3675fcfa03abc21248f...,16124
2,blocks.py,efc9ffe783e0f1b1bd88d7544cf129fb6905ed4afd56f2...,2493
3,config.json,4f4a7f15b8f979b45bfddb165ce9fbbfea9ef7cb5ef05a...,1227
4,configuration_mpt.py,3798eac89b49c0e469fad694cb6abccb12e6a033f2f464...,9080
5,generation_config.json,11e4dba2f93adb5aee189e9aa0dce3d624e72ecf943ac6...,91
6,hf_prefixlm_converter.py,3901fdcbfda941a95deb9b0607db6286894a303714b0a1...,27219
7,meta_init_context.py,8a643f6cd204d93a75a55a200d0f48388f7a1a969b6b0f...,3639
8,modeling_mpt.py,fa3f4937fba59b537153aec0e12c4a376a5ce1ad5fd086...,17406
9,norm.py,275182e7981547443077b191e1f7ad41271725d1d9d8ce...,2563


In [34]:
df.to_csv("mpt-7b-instruct_SHA256_HASHES.csv")